In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [2]:
import tensorflow as tf

In [6]:
df0 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Audiobooks_data.csv')
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/market_segmentation.csv')

In [8]:
df2 = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Audiobooks_data.csv', delimiter=',')

In [14]:
inputs0 = df2[:, 1:-1]
targets0 = df2[:, -1]

In [23]:
num_ones = int(np.sum(targets0))
zeros_counter = 0
index_drop = []

for i in range(targets0.shape[0]):
  if targets0[i] == 0:
    zeros_counter += 1
    if zeros_counter > num_ones:
      index_drop.append(i)

inputs1 = np.delete(inputs0, index_drop, axis=0)
targets1 = np.delete(targets0, index_drop, axis=0)

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler()
scaler.fit(inputs1)

StandardScaler()

In [26]:
inputs2 = scaler.transform(inputs1)

In [28]:
from sklearn.model_selection import train_test_split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(inputs2, targets1, test_size = 0.2, random_state = 42)
X_train, X_val , y_train, y_val = train_test_split(inputs2, targets1, test_size = 0.25, random_state = 42)
#80% == 0.8
# 0.25 * 0.80 = 0.2
# 60 20 20

In [37]:
np.savez('audiobooks_train', inputs=X_train, targets=y_train)
np.savez('audiobooks_test', inputs=X_test, targets=y_test)
np.savez('audiobooks_val', inputs=X_val, targets=y_val)

In [38]:
npz1 = np.load('/content/audiobooks_train.npz')
npz2 = np.load('/content/audiobooks_test.npz')
npz3 = np.load('/content/audiobooks_val.npz')

In [40]:
train_inputs = npz1['inputs'].astype(float)
train_targets = npz1['targets'].astype(float)

test_inputs = npz2['inputs'].astype(float)
test_targets = npz2['targets'].astype(float)

val_inputs = npz3['inputs'].astype(float)
val_targets = npz3['targets'].astype(float)

In [41]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

EPOCHS = 50
BATCH_SIZE = 100
EARLY_STOP = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, train_targets, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          callbacks = EARLY_STOP,
          validation_data = (val_inputs, val_targets))



Epoch 1/50
34/34 [==============================] - 2s 14ms/step - loss: 0.6045 - accuracy: 0.6587 - val_loss: 0.5113 - val_accuracy: 0.7534
Epoch 2/50
34/34 [==============================] - 0s 4ms/step - loss: 0.4731 - accuracy: 0.7586 - val_loss: 0.4387 - val_accuracy: 0.7453
Epoch 3/50
34/34 [==============================] - 0s 6ms/step - loss: 0.4211 - accuracy: 0.7747 - val_loss: 0.4069 - val_accuracy: 0.7802
Epoch 4/50
34/34 [==============================] - 0s 4ms/step - loss: 0.3952 - accuracy: 0.7869 - val_loss: 0.3868 - val_accuracy: 0.7962
Epoch 5/50
34/34 [==============================] - 0s 6ms/step - loss: 0.3819 - accuracy: 0.7943 - val_loss: 0.3744 - val_accuracy: 0.7998
Epoch 6/50
34/34 [==============================] - 0s 7ms/step - loss: 0.3695 - accuracy: 0.8060 - val_loss: 0.3690 - val_accuracy: 0.7927
Epoch 7/50
34/34 [==============================] - 0s 3ms/step - loss: 0.3640 - accuracy: 0.8152 - val_loss: 0.3609 - val_accuracy: 0.7945
Epoch 8/50
34/34 [=

In [42]:
test_loss, test_acc = model.evaluate(test_inputs, test_targets)

28/28 [==============================] - 0s 1ms/step - loss: 0.3443 - accuracy: 0.8067


In [43]:
print('Test_loss: {0: .2f}, Test_accuracy: {1: .2f}%'.format(test_loss, test_acc*100))

Test_loss:  0.34, Test_accuracy:  80.67%
